# Milestone 6: OLS Model with MLRATE
## Estée Lauder MLRATE Adjusted ATE Calculation
## Authors: Selena Ho, Sahiti Srikakolapu

In [ ]:
# Import libraries and packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score
from statsmodels.stats.power import TTestIndPower
import statsmodels.api as sm

In [ ]:
# Load dataset
df = pd.read_parquet("https://drive.google.com/uc?export=download&id=1qTcf1HueAbq9X2PkAwEBTsLPhhyu30YR")

print("Dataset loaded successfully!")
print(f"Shape: {df.shape}")
print(df.head())

# Prepare target and features
y = df['revenue (t)']
#X = df.drop(columns=['revenue (t)', 'customer_id', 'name'], axis=1) # bug: includes T!
X = df.drop(columns=['revenue (t)', 'customer_id', 'name','assignment'], axis=1) # fixed: make sure to use only pre-treatment variables
T = df['assignment']

Dataset loaded successfully!
Shape: (5577, 8)
                                customer_id             name  aov (t-1)  \
18951  be17d8f7-7bb0-40ad-90cd-d4ebb4f5aa51   Christine Sims     134.23   
19248  a4435752-02b4-47c0-8681-9f9b9926dbcc     Billy Orozco      87.62   
323    2b1cfeb3-d356-40bd-93e4-165423c8144e  Stephanie Davis      95.04   
16141  7c961398-d620-43c7-a6e9-60942dd91c78    Angela Miller      52.51   
9276   7e69aa44-a295-49a0-a7f8-9c77bba8d1f8  Michael Ramirez      68.77   

       days_since_last_purchase (t-1)  tenure_in_days(t-1)  \
18951                              34                  102   
19248                               7                   36   
323                                 0                   72   
16141                               4                   18   
9276                               72                   52   

       loyalty_membership  revenue (t)  assignment  
18951                   0   166.233993           1  
19248                   

In [ ]:
def out_of_fold_predict_baseline(X, y, model="RandomForest"):
    """
    Generate out of fold predictions for y using specified model (defaults to RandomForest)
    """
    kf = KFold(n_splits=2, random_state=42, shuffle=True) #creates 2-fold cross-validation splitter
    predictions = np.zeros(len(y)) # array to store resulting predictions

    #iterating over each fold
    for train_idx, test_idx in kf.split(X.values, y.values):
      # extracting training and test subsets for this fold:
      X_train, X_test = X.values[train_idx], X.values[test_idx]
      y_train = y.values[train_idx]

      model = RandomForestRegressor(n_estimators=200, random_state=42, n_jobs=-1) # baseline model initialized
      model.fit(X_train, y_train) # fit the model on the training subset

      predictions[test_idx] = model.predict(X_test) # predict on the test subset and store result in predictions

    return predictions


In [ ]:
# 1) Obtain out-of-fold predictions of baseline outcome
# Reuse your K-fold routine to get leakage-free predictions:
g_hat = out_of_fold_predict_baseline(X, y, model="RandomForest")

# 2) Center the predictions for interpretation & stability
Gbar = g_hat.mean() # “G_mean” in the slides
G_centered = g_hat - Gbar # defines “average baseline” as zero so that T coefficient is the ATE at average baseline

# 3) Build the regression design for ATE with variance reduction
X_reg = sm.add_constant(np.column_stack([T, g_hat, T * G_centered]))

# 4) Fit with robust SEs
model = sm.OLS(y, X_reg).fit(cov_type="HC0") # fit Ordinary Least Squares (OLS) regression model using y as the outcome and X_reg as predictors.

#print summary
print(model.summary(xname=['const', 'T', 'g_hat', 'T*(g_hat-g_bar)']))

                            OLS Regression Results                            
Dep. Variable:            revenue (t)   R-squared:                       0.670
Model:                            OLS   Adj. R-squared:                  0.669
Method:                 Least Squares   F-statistic:                     3653.
Date:                Fri, 14 Nov 2025   Prob (F-statistic):               0.00
Time:                        21:38:02   Log-Likelihood:                -22597.
No. Observations:                5577   AIC:                         4.520e+04
Df Residuals:                    5573   BIC:                         4.523e+04
Df Model:                           3                                         
Covariance Type:                  HC0                                         
                      coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------
const               3.9600      1.591     

In [ ]:
# Extract key results
ate = model.params[1]  # Coefficient on assignment (treatment effect)
ci_lower, ci_upper = model.conf_int().iloc[1]  # 95% CI for treatment coefficient
ci_width = ci_upper - ci_lower
p_value = model.pvalues[1]  # p-value for treatment coefficient
std_error = model.bse[1]  # Standard error

# Display extracted results
print("="*80)
print("Summary:")
print("="*80)
print("Average Treatment Effect (ATE): $",round(ate,2))
print("95% Confidence Interval: [$",round(ci_lower,2),", ","$",round(ci_upper,2),"]")
print("CI Width: $",round(ci_width,2))
print("Standard Error: $",round(std_error,2))
print("P-value: ",p_value)
print(f"Statistical Significance (α=0.05): {'Yes' if p_value < 0.05 else 'No'}")
print(f"CI Includes Zero: {'Yes' if ci_lower <= 0 <= ci_upper else 'No'}")
print("="*80)

Summary:
Average Treatment Effect (ATE): $ 5.66
95% Confidence Interval: [$ 4.93 ,  $ 6.39 ]
CI Width: $ 1.46
Standard Error: $ 0.37
P-value:  4.958732944961112e-52
Statistical Significance (α=0.05): Yes
CI Includes Zero: No


/tmp/ipython-input-448460464.py:2: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  ate = model.params[1]  # Coefficient on assignment (treatment effect)
/tmp/ipython-input-448460464.py:5: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  p_value = model.pvalues[1]  # p-value for treatment coefficient
/tmp/ipython-input-448460464.py:6: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  std_error = model.bse[1]  # Standard error
